# DMPlex from GMsh with labels

In [1]:
import warnings
warnings.filterwarnings("ignore")
import gmsh
gmsh.initialize()
import numpy as np

In [2]:
# By default, if physical groups are defined, Gmsh will export in
# the output mesh file only those elements that belong to at least one physical
# group. To force Gmsh to save all elements, you can use
gmsh.option.setNumber("Mesh.SaveAll", 1)

In [3]:
gmsh.model.add("Test")

## Create a simple unstructured rectangular mesh

In [4]:
gmsh.model.occ.add_rectangle(0, 0, 0, 1.0, 1.0)
gmsh.model.occ.synchronize()

In [5]:
# Get All boundaries
walls = gmsh.model.getEntities(1)
cl = gmsh.model.occ.add_curve_loop(list([wall[1] for wall in walls]))
surface = gmsh.model.occ.add_plane_surface([cl])

## Label all walls

In [6]:
left_marker, right_marker, bottom_marker, top_marker = 1, 3, 5, 7

for wall in walls:
    com = gmsh.model.occ.getCenterOfMass(wall[0], wall[1])
    
    # Left Boundary
    if np.allclose(com, [0., 0.5, 0.]):
        name = "Left"
        marker = left_marker
        
    # Right Boundary
    elif np.allclose(com, [1.0, 0.5, 0.]):
        name = "Right"
        marker = right_marker
        
    # Bottom Boundary
    elif np.allclose(com, [0.5, 0., 0.]):
        name = "Bottom"
        marker = bottom_marker
       
    # Top Boundary
    elif np.allclose(com, [0.5, 1.0, 0.]):
        name = "Top"
        marker = top_marker
        
    gmsh.model.addPhysicalGroup(wall[0], [wall[1]], marker)
    gmsh.model.setPhysicalName(wall[1], marker, name)


In [7]:
walls

[(1, 1), (1, 2), (1, 3), (1, 4)]

In [8]:
gmsh.model.occ.synchronize()
gmsh.model.mesh.generate(2)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 50%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.000308415s, CPU 0.000648s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0111985s, CPU 0.011812s)
Info    : 164 nodes 360 elements


In [9]:
gmsh.write("mesh_test.msh")

Info    : Writing 'mesh_test.msh'...
Info    : Done writing 'mesh_test.msh'


In [10]:
#gmsh.fltk.run()

In [11]:
gmsh.finalize()

# Import Mesh into PETSc using DMPlex

In [12]:
import sys,petsc4py
petsc4py.init(sys.argv)
from petsc4py import PETSc
import numpy as np

In [13]:
options = PETSc.Options()

In [14]:
options["dm_plex_separate_marker"] = None

In [15]:
plex = PETSc.DMPlex().createFromFile("mesh_test.msh")

In [16]:
plex.view()

DM Object: DM_0x558ecba4a580_0 1 MPI processes
  type: plex
DM_0x558ecba4a580_0 in 2 dimensions:
  0-cells: 164
  1-cells: 486
  2-cells: 324
Labels:
  celltype: 3 strata with value/size (0 (164), 3 (324), 1 (486))
  depth: 3 strata with value/size (0 (164), 1 (486), 2 (324))
  Face Sets: 4 strata with value/size (5 (8), 3 (8), 7 (8), 1 (8))


In [17]:
markers_dict = {"Bottom": bottom_marker,
                "Right": right_marker,
                "Top": top_marker,
                "Left": left_marker}

In [18]:
for key, value in markers_dict.items():
    indexSet = plex.getStratumIS("Face Sets", value)
    plex.createLabel(key)
    label = plex.getLabel(key)
    if indexSet:
        label.insertIS(indexSet, value)
    indexSet.destroy()

In [20]:
plex.view()

DM Object: DM_0x558ecba4a580_0 1 MPI processes
  type: plex
DM_0x558ecba4a580_0 in 2 dimensions:
  0-cells: 164
  1-cells: 486
  2-cells: 324
Labels:
  celltype: 3 strata with value/size (0 (164), 3 (324), 1 (486))
  depth: 3 strata with value/size (0 (164), 1 (486), 2 (324))
  Face Sets: 4 strata with value/size (5 (8), 3 (8), 7 (8), 1 (8))
  Bottom: 1 strata with value/size (5 (8))
  Right: 1 strata with value/size (3 (8))
  Top: 1 strata with value/size (7 (8))
  Left: 1 strata with value/size (1 (8))
